# Feedforwad NN for wikt challenge

by Andrej Hucko

Tento model sme nestihli odovzdat do sutaze. Dava velmi dobre vysledky.
Venoval som tomu najviac casu s pomedzi ostatnych hodnotenych casti 2_projektu kedze na BP sa budem venovat neuronovym sietiam 
a tento model mal sluzit ako moj uvod do Tensorflow, implementacie a optimalizacie neuronovych sieti pre urcite problemy.

Pouzili sme jednoduchu doprednu siet s dvomi vrstvami. 2048 skrytych neuronov v kazdej skrytej vrstve. Ako vystup s output layer
sme pouzili tradicnu softmax funkciu.

#### Parametre:
learning_rate = 0.001
reg_constant = 0.001
batch_size = 64
epochs = 100

Namiesto GradientDescent sme pouzili AdamOptimizer, ktory dokaze pocas trenovania dynamicky upravovat hodnotu learning rate.
Ako aktivacnu funkciu sme zvolili ReLU.
Ako nasu chybovu funkciu sme zvolili krizovu entropiu.
Data boli normalizovane pre spracovanim.
Ako vyhodnocovaciu metriku sme zvolili accuracy. A vyhodnocovali sme ju na validacnej vzorke.

#### Tensorboard

pomocou prikazu v cmd: "tensorboard --logdir=./logs/prediction"

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [8]:
class Data:
    def __init__(self, batch_size):
        self.scaler = StandardScaler()
        self.x = Data.import_data('x_train')
        self.y = Data.convert(Data.import_data('y_train'))
        self.val_x = Data.import_data('x_validation')
        self.val_y = Data.convert(Data.import_data('y_validation'))
        self.test_x = Data.import_data('x_test')
        self.pointer = 0
        self.batch_size = batch_size
        
        self.scaler.fit(self.x)
        self.scaler.transform(self.x)
        self.scaler.transform(self.val_x)
        self.scaler.transform(self.test_x)
    
    def shuffle_data(self):
        self.x, self.y = shuffle(self.x, self.y, random_state=0)
    
    def next_batch(self):
        batch = (self.x[self.pointer:self.pointer + self.batch_size], self.y[self.pointer:self.pointer + self.batch_size])
        self.pointer += self.batch_size
        return batch[0], batch[1]
    
    def convert(data):
        return pd.get_dummies(data, columns = [0]).values
        
    def import_data(name):
        return pd.read_csv('../data/'+ name +'.csv', header=None)
    
    def getX(self):
        return self.x
    
    def getTest(self):
        return self.test_x
    
    def valid_data(self):
        return self.val_x
    
    def label_data(self):
        return self.val_y
    
    def move_pointer(self):
        self.pointer = 0
        
    def get_pointer(self):
        return self.pointer
    
    def get_train_size(self):
        return self.x.shape[0]
    
    def get_train_data(self):
        return self.y
    
    def get_scaler(self):
        return self.scaler

In [9]:
# Constants
learning_rate = 0.001
reg_constant = 0.001
#Parameters
epochs = 100
batch_size = 64
logs_path = "./logs"

data = Data(batch_size)

# Network Parameters
n_hidden_1 = 2048 # 1st layer number of neurons
n_hidden_2 = 2048 # 2nd layer number of neurons
num_input = 32 # data input (img shape: 8*4)
num_classes = 2 # total classes (0-1)

with tf.name_scope('input'):
    # Graph inputs
    X = tf.placeholder("float", [None, num_input])
    Y = tf.placeholder("float", [None, num_classes])
    #dropout_prob = tf.placeholder(tf.float32)

In [10]:
# Store layers weight & bias
with tf.name_scope("weights"):
    weights = {
        'h1': tf.Variable(tf.truncated_normal([num_input, n_hidden_1])),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_2, num_classes]))
    }
with tf.name_scope("biases"):
    biases = {
        'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
        'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
        'out': tf.Variable(tf.truncated_normal([num_classes]))
    }

In [11]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 2048 neurons
    with tf.name_scope('hidden_layer_1'):
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        #dropout = tf.nn.dropout(layer_1, dropout_prob)
    # Hidden fully connected layer with 2048 neurons
    with tf.name_scope('hidden_layer_2'):
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        #dropout_2 = tf.nn.dropout(layer_2, dropout_prob)
        
    # Output fully connected layer with a neuron for each class
    with tf.name_scope('ouput_layer_1'):
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    
    return out_layer

In [12]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss_op = tf.reduce_mean(entropy + reg_constant*tf.nn.l2_loss(weights['h1']) +
                         reg_constant*tf.nn.l2_loss(weights['h2']) + reg_constant*tf.nn.l2_loss(weights['out']))

optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [13]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(tf.global_variables_initializer())

    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss_op)
    tf.summary.scalar('learning_rate', learning_rate)
    merged = tf.summary.merge_all()
    
    test_writer = tf.summary.FileWriter(logs_path + '/prediction/' + 'tryhard', sess.graph)
    for step in range(1, epochs):
        while data.get_train_size() > batch_size + data.get_pointer():
            batch_x, batch_y = data.next_batch()
            train.run(feed_dict={X: batch_x, Y: batch_y})
            
        valid_x = data.valid_data()
        valid_y = data.label_data()
        summary, test_acc, loss_val = sess.run([merged, accuracy, loss_op], feed_dict={X: valid_x, Y: valid_y})
        test_writer.add_summary(summary, step)
        print("Epoch: ", step, "Testing Accuracy: ", test_acc, "Loss: ", loss_val)
        
        data.shuffle_data()
        data.move_pointer()
    print("Optimization Finished!")
    
    results = sess.run(logits, feed_dict={X: data.getTest()})

Epoch:  1 Testing Accuracy:  0.8705 Loss:  1755.89
Epoch:  2 Testing Accuracy:  0.88445 Loss:  1665.59
Epoch:  3 Testing Accuracy:  0.89445 Loss:  1414.42
Epoch:  4 Testing Accuracy:  0.85385 Loss:  1356.27
Epoch:  5 Testing Accuracy:  0.9029 Loss:  1170.81
Epoch:  6 Testing Accuracy:  0.88845 Loss:  1154.06
Epoch:  7 Testing Accuracy:  0.9002 Loss:  1011.05
Epoch:  8 Testing Accuracy:  0.9218 Loss:  988.883
Epoch:  9 Testing Accuracy:  0.8181 Loss:  990.998
Epoch:  10 Testing Accuracy:  0.92935 Loss:  827.862
Epoch:  11 Testing Accuracy:  0.9255 Loss:  780.566
Epoch:  12 Testing Accuracy:  0.93585 Loss:  716.612
Epoch:  13 Testing Accuracy:  0.899 Loss:  654.883
Epoch:  14 Testing Accuracy:  0.902 Loss:  636.048
Epoch:  15 Testing Accuracy:  0.93095 Loss:  557.203
Epoch:  16 Testing Accuracy:  0.91575 Loss:  541.112
Epoch:  17 Testing Accuracy:  0.9387 Loss:  515.919
Epoch:  18 Testing Accuracy:  0.92765 Loss:  465.04
Epoch:  19 Testing Accuracy:  0.92405 Loss:  438.212
Epoch:  20 Tes

In [15]:
results.shape

(20000, 2)

In [16]:
out = []
for i in results:
    if i[0] > i[1]:
        out.append(0)
    else:
        out.append(1)

In [17]:
import numpy
a = numpy.asarray(out)
numpy.savetxt("predictions.csv", a, fmt="%d", delimiter=",")